<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intialization" data-toc-modified-id="Intialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intialization</a></span></li><li><span><a href="#Request-API-(Buy)" data-toc-modified-id="Request-API-(Buy)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Request API (Buy)</a></span></li><li><span><a href="#Process-Data" data-toc-modified-id="Process-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Process Data</a></span></li><li><span><a href="#Export-to-csv" data-toc-modified-id="Export-to-csv-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export to csv</a></span></li></ul></div>

In [143]:
import os 
import requests
import json
import re

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

### Intialization

In [140]:
nestoria_endpoint = 'https://api.nestoria.co.uk/api'
data_para = {
    "encoding" : 'json',
    "action" : "search_listings",
    "country" : "uk",
    "listing_type" : "buy",
    "place_name" : "somerset",
    "number_of_results" :15,
    "page" : 5
}p

In [141]:
data_house = requests.get(nestoria_endpoint, data_para)
print(json.loads(data_house.content)['response']['page'])
json_normalize(json.loads(data_house.content)['response']['listings'])[['title', 
                                                                               'price', 'summary']]

5


,title,price,summary
0,"Hillgrove Road, Priddy, Nr Wells, Somerset, BA5",280000,35.2 acres of agricultural land at hillgrove r...
1,"Newton Road, North Petherton, Bridgwater, Some...",395000,An ly presented attached period cottage set in...
2,"Church Lane, Hutton, Weston-super-Mare, North ...",330000,Detached house. Three/four beds. Two/three rec...
3,"Brimgrove Lane, Shepton Beauchamp, Ilminster, ...",400000,A ly presented detached four bedroom home in a...
4,"Crawter Drive, Porlock, Minehead, Somerset, TA24",212500,A well maintained three bedroom semi detached ...
5,"Bishopswood, Chard, Somerset, TA20",749000,A detached farmhouse situated in a rural locat...
6,"Burnham Level, Burnham On Sea, Somerset, TA9",65000,9.04 acres of agricultural pasture land on bur...
7,"Agricultural Pasture Land, Pillmoor Lane, Isle...",130000,13.56 acres of agricultural pasture land strat...
8,"St. Thomas Mews, North Road, Wells, Somerset, BA5",359950,"No onward chain. A modern and townhouse, conve..."
9,"Bishopswood, Chard, Somerset, TA20",375000,A 3 bedroom detached house with glorious views...


In [518]:
json.loads(data_house.content)['response']

{'application_response_code': '100',
 'application_response_text': 'listings returned for one unambiguous location',
 'attribution': {'img_height': 22,
  'img_url': 'https://resources.nestimg.com/nestoria/img/pbr_v1.png',
  'img_width': 183,
  'link_to_img': 'http://www.nestoria.com'},
 'created_http': 'Mon, 17 Jun 2019 15:16:21 GMT',
 'created_unix': 1560784581,
 'link_to_url': 'https://www.nestoria.co.uk/somerset/property/buy/100',
 'listings': [{'bathroom_number': '',
   'bedroom_number': 1,
   'car_spaces': 0,
   'commission': 0,
   'construction_year': 0,
   'datasource_name': 'OnTheMarket.com',
   'img_height': 300,
   'img_url': 'https://imgs.nestimg.com/1_bedroom_retirement_property_for_sale_111043909696632194.jpg',
   'img_width': 400,
   'keywords': 'No Chain, Kitchen',
   'latitude': 50.9403,
   'lister_url': 'https://www.nestoria.co.uk/detail/0000000111043909696632194/title/5/100-1?serpUid=&pt=1&ot=1&l=somerset&did=105_high1&t_sec=9&t_or=45&t_pvid=null&utm_source=api&utm_me

### Request API (Buy)

In [4]:
# head = np.array([
#     'title', 'property_type'
#     'price', 'price_currency', 'price_formatted', 'price_high', 'price_low', 
#     'price_type', 'commission', 'size', 'size_type', 'construction_year', 
#     'keywords', 'bathroom_number', 'bedroom_number', 'car_spaces', 'summary'
#     'latitude', 'longitutde', 'updated_in_days', 'page'])

# head_to_drop = np.array(['datasource_name', 'lister_url', 'thumb_height', 'thumb_url', 
#                          'thumb_width', 'img_height', 'img_url', 'img_width', 
#                          'size', 'size_type', 'price_formatted', 'updated_in_days_formatted'])
# len(head_to_drop)

In [24]:
def check_API(status_code):
    # Check API has worked
    if status_code == 200:
        print("200 API of Nestoria requested successfully!")
    elif status_code == 400:
        print("400 Bad request")
    elif status_code == 403:
        print("403 API of Nestoria call was forbidden. Reached maximum calls")
    else:
        print(f"Misterous error : {status_code}")
        
def load_json_data(endpoint, para, cur_page):
    """
    @endpoint : nestoria API endpiont
    @para : API parameters
    @cur_page : current page to extract
    """
    para['page'] = cur_page
    res_data = requests.get(url= endpoint, params= para)
    check_API(res_data.status_code)
    json_data = json.loads(res_data.content)
    page_num = json_data['request']['page']
    return json_data, page_num
    
def extract_data_source(json_data):
    """
    Build data source, like country, location, created_unix
    """
    cols_request = np.array(['country', 'location', 'page', 'product_type', 
                         'property_type', 'sort'])
    df = json_normalize(json_data['request'])[cols_request]
    df['created_unix'] = json_data['response']['created_unix']
    return df, cols_request

def extract_df_listing(json_data):
    """
    House information is in the json_data['response']['listings']
    """
    cols_head = np.array([
        'title', 'property_type', 'lister_name', 'listing_type',
        'price', 'price_currency', 'price_high', 'price_low', 
        'price_type', 'commission', 'latitude', 'longitude', 'location_accuracy',
        'construction_year', 'keywords', 'bathroom_number', 'bedroom_number', 'car_spaces', 
        'summary', 'updated_in_days'])
    df = json_normalize(json_data['response']['listings'])[cols_head]
    df['page'] = json_data['response']['page']
    return df, cols_head

def get_one_page_data(endpoint, data_para, cur_page= 1):
    """
    @endpoint : API endpoint
    @data_para : API parameters
    @cur_page : current page data to request
    """
    data_para.update({'page' : cur_page})
    json_data, page_num = load_json_data(nestoria_endpoint, data_para, cur_page)
#     print(f'cur_page :{cur_page} page_num: {page_num}')
    if page_num < cur_page:
        print('Input page number too larger!')
        return
    df_data_source, cols_reqeust = extract_data_source(json_data)
    df_listings, cols_head = extract_df_listing(json_data)
    return pd.merge(df_data_source, df_listings, on= 'page', how= 'inner')

def get_all_pages_data(endpoint, data_para, cur_page= 1):
    """
    @endpoint : API endpoint
    @data_para : API parameters
    @cur_page : current page data to request
    API Bug !!! : 
        If request page number is larger than the maximum page number, 
        the API will set the requesting page number as the maximum page number.
        And the total_pages attribute does not match the real maximum page number....
        e.g: If request page numbe 120, while the maximum page numbe is 100, then the API
            will return a response from the data in page 100.
    """
    while True:
        data_para.update({'page' :  cur_page})
        print(f'extracting page {cur_page}...')
        json_data, page_num = load_json_data(nestoria_endpoint, data_para, cur_page)
        if page_num < cur_page:
            print('Extracted Done!')
            break
        df_data_source, cols_reqeust = extract_data_source(json_data)
        df_listings, cols_head = extract_df_listing(json_data)
        print(f'page {page_num} got {df_listings.shape[0]} houses')
        df_house = pd.merge(df_data_source, df_listings, on='page', how='inner')
        if page_num == 1:
            # Initialize a dataframe
            df = df_house
        else:
            df = pd.concat([df, df_house], ignore_index=True)
        cur_page = page_num + 1
    return df

In [20]:
# df_house = get_one_page_data(nestoria_endpoint, data_para, 123)
# df_house.head(3)[['page', 'title', 'lister_name', 'price', 'keywords']]

200 API of Nestoria requested successfully!
Input page number too larger!


In [25]:
df_house = get_all_pages_data(nestoria_endpoint, data_para, 1)

extracting page 1...
200 API of Nestoria requested successfully!
page 1 got 15 houses
extracting page 2...
200 API of Nestoria requested successfully!
page 2 got 15 houses
extracting page 3...
200 API of Nestoria requested successfully!
page 3 got 15 houses
extracting page 4...
200 API of Nestoria requested successfully!
page 4 got 15 houses
extracting page 5...
200 API of Nestoria requested successfully!
page 5 got 15 houses
extracting page 6...
200 API of Nestoria requested successfully!
page 6 got 15 houses
extracting page 7...
200 API of Nestoria requested successfully!
page 7 got 15 houses
extracting page 8...
200 API of Nestoria requested successfully!
page 8 got 15 houses
extracting page 9...
200 API of Nestoria requested successfully!
page 9 got 15 houses
extracting page 10...
200 API of Nestoria requested successfully!
page 10 got 15 houses
extracting page 11...
200 API of Nestoria requested successfully!
page 11 got 15 houses
extracting page 12...
200 API of Nestoria requeste

200 API of Nestoria requested successfully!
page 95 got 15 houses
extracting page 96...
200 API of Nestoria requested successfully!
page 96 got 15 houses
extracting page 97...
200 API of Nestoria requested successfully!
page 97 got 15 houses
extracting page 98...
200 API of Nestoria requested successfully!
page 98 got 15 houses
extracting page 99...
200 API of Nestoria requested successfully!
page 99 got 15 houses
extracting page 100...
200 API of Nestoria requested successfully!
page 100 got 15 houses
extracting page 101...
200 API of Nestoria requested successfully!
Extracted Done!


### Process Data 

In [ ]:
def extract_town(df):
    """
    Extract town name from the title column
    """
    

In [91]:
df_house['dt'] = pd.to_datetime(df_house['created_unix'], unit= 's')
df_house['latitude_longtitude'] = df_house[['latitude', 'longitude']].to_dict('records')

### Export to csv

In [96]:
file_path = os.path.join('.', 'data', 'house_sell_nestoria.csv')
df_house.to_csv(file_path, index= False)

In [100]:
df_test = pd.DataFrame(df_house)

In [121]:
somerset = np.array(['Bath', 'Bridgwater', 'Burnham-On-Sea', 'Chard', 'Cheddar',
                    'Clevedon', 'Frome', 'Glastonbury', 'Ilminster', 'Minehead',
                    'Radstock', 'Shepton Mallet', 'Street', 'Taunton', 'Wellington',
                     'Wells', 'Weston-Super-Mare', 'Wincanton', 'Yeovil'])
somerset = [x.lower() for x in somerset]

In [114]:
df_test.head()

,country,location,page,product_type,property_type_x,sort,created_unix,title,property_type_y,lister_name,...,location_accuracy,construction_year,keywords,bathroom_number,bedroom_number,car_spaces,summary,updated_in_days,dt,latitude_longtitude
0,uk,somerset,1,realestate,property,nestoria_rank,1560787331,"Queen Square, North Curry, Taunton, Somerset, TA3",house,Stags,...,9,0,"Detached, Garden, No Chain, Double Bedroom, Pa...",,3,0,A most village house situated in a conservatio...,2,2019-06-17 16:02:11,"{'latitude': 51.0228, 'longitude': -2.97205}"
1,uk,somerset,1,realestate,property,nestoria_rank,1560787331,"Manor Farm, Forton, Chard, Somerset, TA20",house,Greenslade Taylor Hunt,...,9,0,"Detached, Freehold, Grade II, Conversion, Listed",,5,0,A contemporary conversion of this grade ii lis...,1,2019-06-17 16:02:11,"{'latitude': 50.8586, 'longitude': -2.94579}"
2,uk,somerset,1,realestate,property,nestoria_rank,1560787331,"Whittington Drive, Somerset",house,NaN,...,9,0,"Detached, New Build, No Chain, Conservatory, K...",,2,0,One off design detached bungalow with accommod...,1,2019-06-17 16:02:11,"{'latitude': 51.3604, 'longitude': -2.93822}"
3,uk,somerset,1,realestate,property,nestoria_rank,1560787331,"Vivary Gate, Upper High Street, Taunton, Somer...",flat,Greenslade Taylor Hunt,...,9,0,,,2,0,A top floor 2 bedroom flat situated within thi...,2,2019-06-17 16:02:11,"{'latitude': 51.0117, 'longitude': -3.10403}"
4,uk,somerset,1,realestate,property,nestoria_rank,1560787331,"Farriers Green, Monkton Heathfield, Taunton, S...",house,NaN,...,9,0,"Detached, No Chain, Modern, Gas Central Heating",,3,0,Positioned within a highly sought-after develo...,3,2019-06-17 16:02:11,"{'latitude': 51.0328, 'longitude': -3.06172}"


In [128]:
for i in 'Hampstead Drive, Bridgwater, Somerset, TA6'.split(','):
    i = i.strip().lower()
    if i in somerset:
        print(i)

bridgwater


In [162]:
p1 = r'(somerset)'
regex1 = re.compile(p1)

In [168]:
regex1.search('north somerset')

<re.Match object; span=(6, 14), match='somerset'>

In [172]:
'Griffen Road, Weston-Super-Mare, North Somerset, BS24'.split(',')[2]

' North Somerset'

In [171]:
for place in 'Griffen Road, Weston-Super-Mare, North Somerset, BS24'.split(','):
    print(place.strip().lower())
    place = place.strip().lower()
    if not regex1.search(place):
        print(place)
    else:
        print(f'good one: {place}')

g
g
r
r
i
i
f
f
f
f
e
e
n
n


r
r
o
o
a
a
d
d
,
,


w
w
e
e
s
s
t
t
o
o
n
n
-
-
s
s
u
u
p
p
e
e
r
r
-
-
m
m
a
a
r
r
e
e
,
,


n
n
o
o
r
r
t
t
h
h


s
s
o
o
m
m
e
e
r
r
s
s
e
e
t
t
,
,


b
b
s
s
2
2
4
4


In [169]:
count = 0
got = False
for location in df_test['title']:
    got = False
    for place in location.split(','):
        place = place.strip().lower()
        if regex1.search(place):
            count += 1
            got = True
    if not got:
        print(location)

St Quintins Park, Bathpool - Detached
Wilstock Village, North Petherton
Moravia Close, Bridgwater - No Chain
Middle Stream Close, Bridgwater
Laxton Close, Taunton - Garden
Standish Street, Bridgwater
Buckland Road, Taunton - Garden
Mount Street, Taunton - Garden
Bossington Drive, Priorswood
Flat for sale, FRENCH WEIR - Garden
Sandford Hill, Wembdon
Killams Park, Taunton TA1 - Detached
Thomas Place, Wellington
Langdon Close, Chard - Cul-de-Sac
West Buckland, Wellington - Detached
Caraway Close, Chard - Garden
Teasel Path, Bridgwater - Detached
Langley Marsh, Wiveliscombe - Garden
Hamilton Road, Taunton - Detached
North Petherton, Nr. Bridgwater
Front Street, Chedzoy, Bridgwater
Fore Street, Holcombe Rogus, Wellington, Devon, TA21
House for sale, North Newton - Garden
House for sale, BLEADON HILL - Garden
STAPLEGROVE PARISH - Semi-Detached
Etsome Terrace, Somerton
Horsey Road, Bawdrip, Bridgwater
Lavinia Way, Bridgwater - Balcony
Crestfield Avenue, Bridgwater
House for sale, NERROLS FARM

In [170]:
count

1339

In [173]:
df_test.columns

Index(['country', 'location', 'page', 'product_type', 'property_type_x',
       'sort', 'created_unix', 'title', 'property_type_y', 'lister_name',
       'listing_type', 'price', 'price_currency', 'price_high', 'price_low',
       'price_type', 'commission', 'latitude', 'longitude',
       'location_accuracy', 'construction_year', 'keywords', 'bathroom_number',
       'bedroom_number', 'car_spaces', 'summary', 'updated_in_days', 'dt',
       'latitude_longtitude'],
      dtype='object')